In [76]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub

# print(tf.__version__)

1.9.0


Download Dataset from http://thinknook.com/twitter-sentiment-analysis-training-corpus-dataset-2012-09-22/

In [68]:
dataset = pd.read_csv('/home/exacon02/Desktop/XSent/Sentiment Analysis Dataset.csv',error_bad_lines = False)
dataset = dataset[["SentimentText","Sentiment"]]
tweets=dataset['SentimentText']
sent=dataset['Sentiment']

train_size = int(len(tweets) * .8)

train_tweets = tweets[:train_size]
train_sent = sent[:train_size]

test_tweets = tweets[train_size:]
test_sent = sent[train_size:]

b'Skipping line 8836: expected 4 fields, saw 5\n'
b'Skipping line 535882: expected 4 fields, saw 7\n'


In [69]:
tweet_embeddings = hub.text_embedding_column(
  "tweets", 
  module_spec="https://tfhub.dev/google/universal-sentence-encoder/2"
)

In [71]:
estimator = tf.estimator.DNNClassifier(
hidden_units = [128, 24],
feature_columns = [tweet_embeddings],
n_classes = 2,
optimizer = tf.train.AdamOptimizer(learning_rate=0.003),
model_dir='/path/to/model/'
#Path to Directory Where Model is to be Stored
)

In [72]:
features = {
  "tweets": np.array(train_tweets)
}
labels = np.array(train_sent)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    features, 
    labels, 
    shuffle=True, 
    batch_size=512, 
    num_epochs=10
)

In [73]:
tf.logging.set_verbosity(tf.logging.ERROR)
estimator.train(input_fn=train_input_fn)

In [74]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"tweets": np.array(test_tweets)}, np.array(test_sent), shuffle=False)

estimator.evaluate(input_fn=eval_input_fn)

{'accuracy': 0.8089528,
 'accuracy_baseline': 0.55986416,
 'auc': 0.8894448,
 'auc_precision_recall': 0.8673041,
 'average_loss': 0.4150449,
 'label/mean': 0.4401358,
 'loss': 53.116833,
 'precision': 0.78977853,
 'prediction/mean': 0.44788742,
 'recall': 0.7712164,
 'global_step': 24666}

In [75]:
while 1:
    evalSentence =input('Input a sentence to be evaluated, or Enter to quit: ')

    if len(evalSentence) == 0:
        break

        
    Sentiment=["Negative","Positive"]
    # Format your input for the neural net
    evalSentence=[evalSentence]
    predict_input_fn = tf.estimator.inputs.numpy_input_fn({"tweets": np.array(evalSentence)}, shuffle=False)
    results = estimator.predict(predict_input_fn)   
    for x in results:
        pred=x['probabilities']
        print(Sentiment[np.argmax(pred)],"sentiment;",pred[np.argmax(pred)] * 100,"% confidence")

Input a sentence to be evaluated, or Enter to quit: He is happy
Positive sentiment; 98.29110503196716 % confidence
Input a sentence to be evaluated, or Enter to quit: He is sad
Negative sentiment; 99.74668025970459 % confidence
Input a sentence to be evaluated, or Enter to quit: He is not happy
Negative sentiment; 95.47165036201477 % confidence
Input a sentence to be evaluated, or Enter to quit: He is extremely happy
Positive sentiment; 95.55400609970093 % confidence
Input a sentence to be evaluated, or Enter to quit: Get out of my house\
Negative sentiment; 79.34272289276123 % confidence
Input a sentence to be evaluated, or Enter to quit: I love her
Positive sentiment; 91.72520041465759 % confidence
Input a sentence to be evaluated, or Enter to quit: I want to stab him
Negative sentiment; 97.03577756881714 % confidence
Input a sentence to be evaluated, or Enter to quit: I want to kill Soham
Negative sentiment; 96.07805609703064 % confidence
Input a sentence to be evaluated, or Enter t